# 04: Gradient and Temporal Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/04_advanced_analysis.ipynb)

This notebook covers advanced snowpack analysis techniques including gradient analysis and temporal patterns.

## What You'll Learn

- Density gradient analysis to identify weak layers
- Temperature gradient analysis for understanding metamorphism
- Temporal analysis with rolling windows and rates of change
- Comparing multiple locations

> **Note**: For advanced xarray techniques like broadcasting, alignment, groupby, and resampling, see **08_advanced_xarray_techniques.ipynb**.


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



In [ ]:
import xsnow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load sample data
print("Loading xsnow sample data...")
try:
    ds = xsnow.single_profile_timeseries()
    print("✅ Data loaded successfully")
    print(f"Dataset dimensions: {dict(ds.dims)}")
except Exception as e:
    print(f"⚠️ Error loading sample data: {e}")
    print("Make sure xsnow is properly installed:")
    print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    raise

%matplotlib inline


## Part 1: Density Gradient Analysis

Density gradients can help identify weak layers in the snowpack. Large changes in density with depth may indicate potential weak layers.


#### XArray Broadcasting (Automatic)

xarray automatically handles broadcasting when working with DataArrays:


In [ ]:
# Example 1: Add a constant to all density values
# xarray automatically broadcasts the scalar to match the array shape
density_plus_10 = ds['density'] + 10  # Adds 10 to every density value
print(f"Original density range: {ds['density'].min().values:.1f} - {ds['density'].max().values:.1f} kg/m³")
print(f"After adding 10: {density_plus_10.min().values:.1f} - {density_plus_10.max().values:.1f} kg/m³")

# Example 2: Normalize density by mean (broadcasting division)
mean_density = ds['density'].mean()
normalized_density = ds['density'] / mean_density
print(f"Normalized density mean: {normalized_density.mean().values:.3f}")

# Example 3: Operations between arrays of different shapes
# Surface density (no layer dimension) vs full density profile
surface_density = ds['density'].isel(layer=0)
density_diff_from_surface = ds['density'] - surface_density
print(f"Density difference from surface shape: {density_diff_from_surface.shape}")


#### NumPy Broadcasting Rules

NumPy has specific broadcasting rules that are important to understand:


In [ ]:
# NumPy broadcasting rules:
# 1. Arrays are aligned from the right
# 2. Dimensions of size 1 are "stretched" to match
# 3. Missing dimensions are treated as size 1

# Example 1: 1D array with scalar
arr_1d = np.array([100, 200, 300, 400])  # Shape: (4,)
scalar = 50
result = arr_1d + scalar  # Broadcasts scalar to (4,)
print(f"1D + scalar: {result}")

# Example 2: 2D array with 1D array
arr_2d = np.array([[100, 200], [300, 400]])  # Shape: (2, 2)
arr_1d = np.array([10, 20])  # Shape: (2,)
result = arr_2d + arr_1d  # Broadcasts (2,) to (1, 2) then to (2, 2)
print(f"2D + 1D:\n{result}")

# Example 3: 3D array with 1D array (like snowpack data)
# Simulate: (time, layer) array with (layer,) array
arr_3d = np.random.rand(10, 5)  # 10 time steps, 5 layers
arr_1d = np.array([0.1, 0.2, 0.3, 0.4, 0.5])  # 5 layers
result = arr_3d + arr_1d  # Broadcasts (5,) to (1, 5) then to (10, 5)
print(f"3D + 1D shape: {result.shape}")


#### Performance: NumPy vs XArray Broadcasting

For large arrays, NumPy operations can be faster, but xarray provides more convenience:


In [ ]:
import time

# Get a subset for timing comparison
density_subset = ds['density'].isel(location=0, slope=0, realization=0)

# Method 1: XArray broadcasting (convenient, preserves metadata)
start = time.time()
result_xarray = density_subset + 10
time_xarray = time.time() - start
print(f"XArray broadcasting: {time_xarray*1000:.3f} ms")

# Method 2: NumPy broadcasting (potentially faster for large arrays)
start = time.time()
density_numpy = density_subset.values
result_numpy = density_numpy + 10
time_numpy = time.time() - start
print(f"NumPy broadcasting: {time_numpy*1000:.3f} ms")

# Note: For small arrays, the difference is negligible
# For very large arrays, NumPy can be faster, but you lose coordinate information
print(f"\nSpeedup: {time_xarray/time_numpy:.2f}x")
print("Note: XArray preserves coordinates and metadata, NumPy is just the raw array")


#### Practical Example: Efficient Computation with Broadcasting

Use broadcasting to efficiently compute operations across dimensions:


In [ ]:
# Example: Normalize each profile by its mean density
# This requires computing mean for each profile, then broadcasting

# Get mean density for each profile (removes 'layer' dimension)
mean_density_per_profile = ds['density'].mean(dim='layer')  # Shape: (location, time, slope, realization)

# Broadcast to subtract from full density array
# xarray automatically broadcasts (location, time, slope, realization) to (location, time, slope, realization, layer)
normalized_density = ds['density'] - mean_density_per_profile

print(f"Original density shape: {ds['density'].shape}")
print(f"Mean per profile shape: {mean_density_per_profile.shape}")
print(f"Normalized density shape: {normalized_density.shape}")
print(f"Normalized mean (should be ~0): {normalized_density.mean().values:.6f}")

# Example 2: Compute anomaly from time mean
# Mean density over time for each location
mean_density_over_time = ds['density'].mean(dim='time')  # Shape: (location, slope, realization, layer)

# Anomaly = current - time mean
density_anomaly = ds['density'] - mean_density_over_time
print(f"\nDensity anomaly shape: {density_anomaly.shape}")
print(f"Anomaly mean: {density_anomaly.mean().values:.3f} kg/m³")


#### When to Use NumPy Broadcasting Directly

Use NumPy broadcasting when:
- **Performance is critical**: Very large arrays where speed matters
- **No coordinate information needed**: You're doing pure numerical computation
- **Integration with other libraries**: Libraries that expect NumPy arrays
- **Custom algorithms**: Complex operations not easily expressed in xarray

Use XArray broadcasting when:
- **Preserve metadata**: You need coordinates, units, and attributes
- **Labeled operations**: You want to use dimension names instead of indices
- **Automatic alignment**: Working with multiple datasets that need alignment
- **Readability**: Code is clearer with labeled dimensions


**Now You Try**: After learning about broadcasting, try:
- Using NumPy broadcasting to normalize density values manually
- Comparing performance of xarray vs NumPy operations on a large subset
- Creating a normalized temperature variable using broadcasting
- Using broadcasting to compute layer-wise differences from surface values


### Alignment: Putting Data on the Same Grid

**Alignment** ensures that when you perform operations between datasets, they're aligned by their coordinates (location, time, etc.). xarray automatically aligns data by matching coordinate values, which is essential when working with multiple data sources or different time periods.

**What you'll see**: The examples below demonstrate how xarray aligns datasets with different coordinate values, ensuring operations are performed on matching data points.


## Part 1.5: Pandas GroupBy Operations

The **groupby** operation is a powerful pattern for analyzing data by groups. It follows a "split-apply-combine" strategy:
1. **Split**: Divide data into groups based on some criteria
2. **Apply**: Perform operations on each group
3. **Combine**: Merge results back together

**What you'll see**: The examples below show how to group snowpack data by location, time periods, or other dimensions to compute statistics and compare groups.

### Understanding GroupBy

GroupBy allows you to:
- Compare statistics across different locations
- Analyze seasonal patterns (group by month)
- Compare different slope aspects
- Aggregate data in meaningful ways


### Grouping by Location

Compare statistics across different locations:


In [ ]:
# Group by location and compute mean density
mean_density_by_location = ds['density'].groupby('location').mean(dim=['time', 'layer'])
print("Mean density by location:")
for loc in mean_density_by_location.location.values:
    mean_dens = mean_density_by_location.sel(location=loc).values
    print(f"  Location {loc}: {mean_dens:.1f} kg/m³")

# Group by location and compute multiple statistics
density_stats = ds['density'].groupby('location').agg(['mean', 'std', 'min', 'max'], dim=['time', 'layer'])
print("\nDensity statistics by location:")
print(density_stats)


### Grouping by Time Periods

Analyze seasonal or monthly patterns:


In [ ]:
# Group by month to see seasonal patterns
# First, ensure time coordinate is datetime
mean_density_by_month = ds['density'].groupby('time.month').mean(dim=['location', 'layer', 'time'])
print("Mean density by month:")
for month in mean_density_by_month.month.values:
    mean_dens = mean_density_by_month.sel(month=month).values
    month_name = pd.Timestamp(2000, month, 1).strftime('%B')
    print(f"  {month_name}: {mean_dens:.1f} kg/m³")

# Group by day of year (1-365) to see annual patterns
mean_density_by_doy = ds['density'].groupby('time.dayofyear').mean(dim=['location', 'layer', 'time'])
print(f"\nMean density by day of year: {len(mean_density_by_doy)} days")
print(f"  Range: {mean_density_by_doy.dayofyear.min().values} - {mean_density_by_doy.dayofyear.max().values}")


### Multiple Grouping Keys

Group by multiple dimensions simultaneously:


In [ ]:
# Group by both location and month
# This gives you mean density for each location in each month
density_by_location_month = ds['density'].groupby(['location', 'time.month']).mean(dim=['layer', 'time'])
print("Mean density by location and month:")
print(f"  Shape: {density_by_location_month.shape}")
print(f"  Dimensions: {density_by_location_month.dims}")

# Access specific location and month
if len(density_by_location_month.location) > 0 and len(density_by_location_month.month) > 0:
    first_loc = density_by_location_month.location.values[0]
    first_month = density_by_location_month.month.values[0]
    value = density_by_location_month.sel(location=first_loc, month=first_month).values
    print(f"  Example: Location {first_loc}, Month {first_month}: {value:.1f} kg/m³")


### Custom Aggregation Functions

Apply custom functions to each group:


In [ ]:
# Define a custom function to compute the range (max - min)
def density_range(x):
    """Compute the range of density values."""
    return x.max(dim=['time', 'layer']) - x.min(dim=['time', 'layer'])

# Apply custom function to each location
density_range_by_location = ds['density'].groupby('location').apply(density_range)
print("Density range (max - min) by location:")
for loc in density_range_by_location.location.values:
    range_val = density_range_by_location.sel(location=loc).values
    print(f"  Location {loc}: {range_val:.1f} kg/m³")

# Another example: count of layers above threshold per location
def count_dense_layers(x, threshold=300):
    """Count layers with density above threshold."""
    return (x > threshold).sum(dim=['time', 'layer'])

dense_layer_count = ds['density'].groupby('location').apply(count_dense_layers, threshold=300)
print("\nCount of dense layers (> 300 kg/m³) by location:")
for loc in dense_layer_count.location.values:
    count = dense_layer_count.sel(location=loc).values
    print(f"  Location {loc}: {count:.0f} layers")


### Practical Example: Comparing Seasonal Patterns

Let's use groupby to compare how density changes seasonally across locations:


In [ ]:
# Group by location and month, compute mean density
seasonal_density = ds['density'].groupby(['location', 'time.month']).mean(dim=['layer', 'time'])

# Visualize seasonal patterns
if len(seasonal_density.location) > 0 and len(seasonal_density.month) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    for loc in seasonal_density.location.values[:3]:  # Show first 3 locations
        monthly_values = [seasonal_density.sel(location=loc, month=m).values 
                         for m in seasonal_density.month.values]
        ax.plot(seasonal_density.month.values, monthly_values, 
               marker='o', label=f'Location {loc}', linewidth=2)
    
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('Mean Density (kg/m³)', fontsize=12)
    ax.set_title('Seasonal Density Patterns by Location', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("✅ Seasonal patterns visualized!")
else:
    print("Note: Insufficient data for seasonal comparison")


**Now You Try**: After learning about groupby operations, try:
- Grouping temperature by month and finding the coldest month
- Grouping by location and computing the standard deviation of density
- Creating a custom function to find the month with the highest snow height for each location
- Grouping by both location and slope to compare different aspects


In [ ]:
# Example 1: Automatic alignment by coordinates
# When you select different subsets, xarray aligns them automatically
ds_location_0 = ds.isel(location=0)
ds_location_1 = ds.isel(location=1)

# Even though these have different location coordinates, operations align automatically
# by matching time, slope, realization, and layer coordinates
density_diff = ds_location_0['density'] - ds_location_1['density']
print(f"Density difference shape: {density_diff.dims}")
print(f"Mean difference: {density_diff.mean().values:.2f} kg/m³")

# Example 2: Alignment when time coordinates differ
# Select different time ranges
times = ds.coords['time'].values
ds_first_half = ds.sel(time=slice(times[0], times[len(times)//2]))
ds_second_half = ds.sel(time=slice(times[len(times)//2], times[-1]))

# Operations between these will align by matching coordinates (location, slope, etc.)
# but time won't match, so the result will have the union of time coordinates
print(f"\nFirst half time range: {ds_first_half.coords['time'].values[0]} to {ds_first_half.coords['time'].values[-1]}")
print(f"Second half time range: {ds_second_half.coords['time'].values[0]} to {ds_second_half.coords['time'].values[-1]}")

# Example 3: Manual alignment using xarray's align function
import xarray as xr

# Create two datasets with slightly different coordinates
# (In practice, this might happen when loading data from different sources)
ds1 = ds.isel(location=0, time=slice(0, 10))
ds2 = ds.isel(location=0, time=slice(5, 15))  # Overlapping but different time range

# Align them to have the same coordinates (intersection by default)
ds1_aligned, ds2_aligned = xr.align(ds1, ds2, join='inner')  # 'inner' = intersection
print(f"\nAfter alignment:")
print(f"ds1_aligned time range: {ds1_aligned.coords['time'].values[0]} to {ds1_aligned.coords['time'].values[-1]}")
print(f"ds2_aligned time range: {ds2_aligned.coords['time'].values[0]} to {ds2_aligned.coords['time'].values[-1]}")

# Now operations are straightforward
density_comparison = ds1_aligned['density'] - ds2_aligned['density']
print(f"Comparison shape: {density_comparison.dims}")

print("\n✅ Key takeaway: xarray automatically handles alignment in most cases!")
print("   You only need manual alignment when working with datasets from different sources.")



**Now You Try**: After learning about broadcasting and alignment, try:
- Creating a normalized density variable by dividing by the mean density across all locations
- Comparing temperature between two different time periods (hint: use `.sel()` with time slices)
- Adding a constant value to all density measurements and seeing how it affects other calculations


## Part 2: Density Gradient Analysis

Density gradients can help identify weak layers in the snowpack. Large changes in density with depth may indicate potential weak layers.


In [ ]:
# Compute density gradient (change in density with depth)
# Large gradients can indicate weak layers
density_gradient = ds['density'].diff(dim='layer')
print(f"Density gradient shape: {density_gradient.shape}")

# Select a single profile to visualize
profile = ds.isel(location=0, time=-1, slope=0, realization=0)
depth = -profile.coords['z'].values
density = profile['density'].values
gradient = density_gradient.isel(location=0, time=-1, slope=0, realization=0).values

# Plot density and gradient
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.plot(density, depth, 'b-', linewidth=2)
ax1.set_xlabel('Density (kg/m³)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('Density Profile')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

ax2.plot(gradient, depth[:-1], 'r-', linewidth=2)
ax2.axvline(x=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Density Gradient (kg/m³ per layer)')
ax2.set_ylabel('Depth (m)')
ax2.set_title('Density Gradient')
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()


In [ ]:
# Compute density gradient (change in density with depth)
# Large gradients can indicate weak layers
density_gradient = ds['density'].diff(dim='layer')
print(f"Density gradient shape: {density_gradient.shape}")

# Select a single profile to visualize
profile = ds.isel(location=0, time=-1, slope=0, realization=0)
depth = -profile.coords['z'].values
density = profile['density'].values
gradient = density_gradient.isel(location=0, time=-1, slope=0, realization=0).values

# Plot density and gradient
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.plot(density, depth, 'b-', linewidth=2)
ax1.set_xlabel('Density (kg/m³)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('Density Profile')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

ax2.plot(gradient, depth[:-1], 'r-', linewidth=2)
ax2.axvline(x=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Density Gradient (kg/m³ per layer)')
ax2.set_ylabel('Depth (m)')
ax2.set_title('Density Gradient')
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()


**Now You Try**: After running the gradient analysis above, try:
- Computing density gradient for a different time step
- Finding layers with density gradient > 50 kg/m³ per layer
- Comparing density gradients between different locations


In [ ]:
# Get snow height time series
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)

# Resample to daily averages (1D = 1 day)
# This averages all values within each day
hs_daily = hs_series.resample(time='1D').mean()
print(f"Original time steps: {len(hs_series.time)}")
print(f"After daily resampling: {len(hs_daily.time)}")
print(f"Time frequency: {hs_daily.time.values[1] - hs_daily.time.values[0]}")

# Compare original vs resampled
times_orig = ds.coords['time'].values
times_daily = hs_daily.time.values

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(times_orig, hs_series.values, 'b-', alpha=0.3, label='Original', linewidth=1)
ax.plot(times_daily, hs_daily.values, 'r-', marker='o', label='Daily average', linewidth=2, markersize=4)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Snow Height: Original vs Daily Resampled', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Resampling to Weekly and Monthly Averages

Resample to longer time periods:


In [ ]:
# Resample to weekly averages (7D = 7 days)
hs_weekly = hs_series.resample(time='7D').mean()
print(f"Weekly resampling: {len(hs_weekly.time)} time steps")

# Resample to monthly averages (1M = 1 month)
hs_monthly = hs_series.resample(time='1M').mean()
print(f"Monthly resampling: {len(hs_monthly.time)} time steps")

# Compare different resampling frequencies
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(times_orig, hs_series.values, 'b-', alpha=0.2, label='Original', linewidth=1)
ax.plot(hs_daily.time.values, hs_daily.values, 'g-', marker='o', label='Daily', linewidth=1.5, markersize=3)
ax.plot(hs_weekly.time.values, hs_weekly.values, 'orange', marker='s', label='Weekly', linewidth=2, markersize=5)
ax.plot(hs_monthly.time.values, hs_monthly.values, 'r-', marker='^', label='Monthly', linewidth=2.5, markersize=6)
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Snow Height: Different Resampling Frequencies', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Different Aggregation Methods

You can use different aggregation methods when resampling:


In [ ]:
# Resample with different aggregation methods
hs_daily_mean = hs_series.resample(time='1D').mean()    # Average
hs_daily_max = hs_series.resample(time='1D').max()      # Maximum
hs_daily_min = hs_series.resample(time='1D').min()      # Minimum
hs_daily_sum = hs_series.resample(time='1D').sum()      # Sum (useful for cumulative variables)

print("Daily resampling with different methods:")
print(f"  Mean: {hs_daily_mean.values[0]:.3f} m")
print(f"  Max:  {hs_daily_max.values[0]:.3f} m")
print(f"  Min:  {hs_daily_min.values[0]:.3f} m")
print(f"  Sum:  {hs_daily_sum.values[0]:.3f} m")

# Visualize mean, max, and min
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(hs_daily_mean.time.values, hs_daily_mean.values, 'b-', label='Daily mean', linewidth=2)
ax.plot(hs_daily_max.time.values, hs_daily_max.values, 'r--', label='Daily max', linewidth=1.5, alpha=0.7)
ax.plot(hs_daily_min.time.values, hs_daily_min.values, 'g--', label='Daily min', linewidth=1.5, alpha=0.7)
ax.fill_between(hs_daily_mean.time.values, hs_daily_min.values, hs_daily_max.values, 
                alpha=0.2, color='gray', label='Daily range')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Daily Snow Height: Mean, Max, and Min', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Resampling Layer-Level Data

You can also resample layer-level variables (like density or temperature):


In [ ]:
# Resample surface layer temperature to daily averages
surface_temp = ds['temperature'].isel(location=0, layer=0, slope=0, realization=0)
temp_daily = surface_temp.resample(time='1D').mean()

print(f"Original temperature time steps: {len(surface_temp.time)}")
print(f"Daily resampled: {len(temp_daily.time)}")

# Resample density for a specific layer
# Get mean density across all layers, then resample
mean_density = ds['density'].mean(dim='layer').isel(location=0, slope=0, realization=0)
density_daily = mean_density.resample(time='1D').mean()

# Visualize
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
times_orig = surface_temp.time.values
times_daily = temp_daily.time.values

ax1.plot(times_orig, surface_temp.values, 'b-', alpha=0.3, label='Original', linewidth=1)
ax1.plot(times_daily, temp_daily.values, 'r-', marker='o', label='Daily average', linewidth=2, markersize=4)
ax1.set_ylabel('Temperature (°C)', fontsize=12)
ax1.set_title('Surface Temperature', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(density_daily.time.values, density_daily.values, 'g-', marker='s', linewidth=2, markersize=4)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel('Mean Density (kg/m³)', fontsize=12)
ax2.set_title('Daily Average Mean Density', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Handling Irregular Time Series

If your data has irregular time spacing, resample can still work:


In [ ]:
# Resample handles irregular time series automatically
# It creates regular time bins and aggregates data within each bin

# Example: Resample to daily, handling missing days
hs_daily_filled = hs_series.resample(time='1D').mean()

# Check for missing days (NaN values)
missing_days = hs_daily_filled.isnull().sum().values
print(f"Missing days in resampled data: {missing_days}")

# Fill missing days using interpolation
hs_daily_interpolated = hs_daily_filled.interpolate_na(dim='time', method='linear')
print(f"After interpolation: {hs_daily_interpolated.isnull().sum().values} missing values")

# Or forward fill (use previous value)
hs_daily_ffill = hs_daily_filled.fillna(method='ffill')
print(f"After forward fill: {hs_daily_ffill.isnull().sum().values} missing values")


### Common Resampling Frequencies

Here are common time frequency strings for resampling:

- `'1H'` - Hourly
- `'1D'` - Daily
- `'7D'` or `'1W'` - Weekly
- `'1M'` - Monthly (calendar month)
- `'1Q'` - Quarterly
- `'1Y'` or `'1A'` - Yearly/Annual

You can also use offsets like `'2D'` (every 2 days) or `'3H'` (every 3 hours).


**Now You Try**: After learning about resampling, try:
- Resampling snow height to weekly averages and comparing with daily averages
- Resampling temperature to monthly means and finding the warmest month
- Using different aggregation methods (max, min, std) when resampling
- Resampling multiple variables simultaneously


In [ ]:
# Temperature gradient = change in temperature per unit depth
# Large gradients (> 10 K/m) can lead to faceting (weak layer formation)

# Compute gradient using z coordinate
temp_diff = ds['temperature'].diff(dim='layer')
z_diff = ds.coords['z'].diff(dim='layer')
temp_gradient = temp_diff / z_diff

# Find layers with high gradients (> 10 K/m indicates faceting)
high_gradient = temp_gradient > 10  # K/m
n_high_gradient = high_gradient.sum().values
print(f"Found {n_high_gradient} layers with temperature gradient > 10 K/m")

# Visualize temperature gradient for a single profile
profile = ds.isel(location=0, time=-1, slope=0, realization=0)
depth = -profile.coords['z'].values
tg_profile = temp_gradient.isel(location=0, time=-1, slope=0, realization=0)
temp_profile = ds['temperature'].isel(location=0, time=-1, slope=0, realization=0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.plot(temp_profile.values, depth, 'b-', linewidth=2)
ax1.set_xlabel('Temperature (°C)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('Temperature Profile')
ax1.grid(True, alpha=0.3)
ax1.invert_yaxis()

ax2.plot(tg_profile.values, depth[:-1], 'r-', linewidth=2)
ax2.axvline(x=10, color='k', linestyle='--', alpha=0.5, label='10 K/m threshold')
ax2.set_xlabel('Temperature Gradient (K/m)')
ax2.set_ylabel('Depth (m)')
ax2.set_title('Temperature Gradient')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()


### Computing Temperature Gradient

First, let's compute the temperature gradient:


## Part 2: Temperature Gradient Analysis

Temperature gradients are important for understanding metamorphism and weak layer formation:


### 📊 Proof by Example, Disproof by Counterexample

**Proof by Example, Disproof by Counterexample Pattern**: Let's explore common misconceptions about temperature gradients and weak layers.

**Example (Proof)**: High temperature gradients (> 10 K/m) often indicate faceting and weak layer formation. Let's see this in action:


In [ ]:
# Example: High temperature gradient leading to weak layer
# Find profiles with high temperature gradients
high_tg_profiles = temp_gradient > 10  # K/m

# Count how many layers have high gradients
n_high_tg = high_tg_profiles.sum().values
print(f"Found {n_high_tg} layers with temperature gradient > 10 K/m")
print("This supports the idea that high gradients can indicate weak layer formation.")

# Show an example profile with high gradient
if n_high_tg > 0:
    # Find first occurrence
    high_tg_locations = high_tg_profiles.any(dim=['time', 'layer'])
    print(f"\nExample: {high_tg_locations.sum().values} locations have high temperature gradients")


**Counterexample (Disproof)**: However, high temperature gradients don't ALWAYS mean weak layers. Sometimes other factors matter more. Let's explore:


In [ ]:
# Counterexample: High gradient but strong layer (due to other factors)
# Example: High gradient with high density might still be strong
high_tg_high_density = (temp_gradient > 10) & (ds['density'] > 300)
n_strong_high_tg = high_tg_high_density.sum().values
print(f"Found {n_strong_high_tg} layers with high temp gradient BUT high density (> 300 kg/m³)")
print("This shows that temperature gradient alone doesn't always indicate weak layers.")
print("Other factors (density, grain type, bonding) also matter!")


**Counterexample (Disproof)**: However, high temperature gradients don't ALWAYS mean weak layers. Sometimes other factors matter more. Let's explore:


In [ ]:
# Counterexample: High gradient but strong layer (due to other factors)
# Example: High gradient with high density might still be strong
high_tg_high_density = (temp_gradient > 10) & (ds['density'] > 300)
n_strong_high_tg = high_tg_high_density.sum().values
print(f"Found {n_strong_high_tg} layers with high temp gradient BUT high density (> 300 kg/m³)")
print("This shows that temperature gradient alone doesn't always indicate weak layers.")
print("Other factors (density, grain type, bonding) also matter!")


**Now You Try**: After running the temperature gradient analysis, try:
- Finding profiles where high temperature gradient (> 10 K/m) occurs with low density (< 150 kg/m³) - these are more likely weak layers
- Comparing temperature gradients between different time periods
- Plotting temperature gradient vs density to see the relationship


### Resample: Time-Based Resampling

**Resample** allows you to resample time series data to different frequencies (daily, weekly, monthly, etc.). This is useful for smoothing data or computing statistics over time periods.

**What you'll see**: The examples below show how to resample time series data to different frequencies and compute statistics.


**Now You Try**: After learning about groupby and resample, try:
- Grouping density by location and computing the standard deviation (hint: use `.std()`)
- Resampling snow height to weekly averages (if your time frequency allows)
- Grouping temperature by month and finding the coldest month


In [ ]:
# Compare mean density across locations
mean_density_by_location = ds['density'].mean(dim=['time', 'layer'])
print("Mean density by location:")
for i, loc in enumerate(ds.coords['location'].values):
    mean_dens = mean_density_by_location.isel(location=i).values
    print(f"  Location {loc}: {mean_dens:.1f} kg/m³")

# Compare snow height evolution
print("\nSnow height by location (last time step):")
for i, loc in enumerate(ds.coords['location'].values):
    hs_loc = ds['HS'].isel(location=i, slope=0, realization=0, time=-1)
    print(f"  Location {loc}: {hs_loc.values:.2f} m")


**Now You Try**: After comparing locations, try:
- Comparing a different metric (e.g., temperature gradient or SWE) across locations
- Finding which location has the most variable density over time
- Creating a visualization comparing multiple locations side-by-side


**Now You Try**: After comparing locations, try:
- Comparing a different metric (e.g., temperature gradient or SWE) across locations
- Finding which location has the most variable density over time
- Creating a visualization comparing multiple locations side-by-side


In [ ]:
# Extract snow height time series
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
times = ds.coords['time'].values

# 7-day rolling average (smoothing)
hs_7day = hs_series.rolling(time=7, center=True).mean()

# Rate of change
hs_rate = hs_series.diff(dim='time')

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(times, hs_series.values, 'b-', alpha=0.5, label='Original')
ax1.plot(times, hs_7day.values, 'r-', linewidth=2, label='7-day rolling average')
ax1.set_xlabel('Time')
ax1.set_ylabel('Snow Height (m)')
ax1.set_title('Snow Height Time Series')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(times[1:], hs_rate.values[1:], 'g-', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Time')
ax2.set_ylabel('Rate of Change (m/time step)')
ax2.set_title('Snow Height Rate of Change')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean rate of change: {hs_rate.mean().values:.4f} m/time step")


**Now You Try**: After running the temporal analysis, try:
- Computing a different rolling window (try 3-day, 14-day, or 30-day averages)
- Calculating the rate of change for a different variable (e.g., density or temperature)
- Finding the time period with the fastest snow height increase


## Part 3: Comparing Multiple Locations

When you have data from multiple locations, you can compare their characteristics.


In [ ]:
import xarray as xr

# Example 1: Aligning datasets with different time ranges
# Split data into two time periods
times = ds.coords['time'].values
mid_point = len(times) // 2

ds_period1 = ds.sel(time=slice(times[0], times[mid_point]))
ds_period2 = ds.sel(time=slice(times[mid_point], times[-1]))

print("Original dataset time range:")
print(f"  Start: {times[0]}")
print(f"  End: {times[-1]}")
print(f"\nPeriod 1 time range: {ds_period1.coords['time'].values[0]} to {ds_period1.coords['time'].values[-1]}")
print(f"Period 2 time range: {ds_period2.coords['time'].values[0]} to {ds_period2.coords['time'].values[-1]}")

# Align them (find intersection of coordinates)
ds1_aligned, ds2_aligned = xr.align(ds_period1, ds_period2, join='inner')
print(f"\nAfter alignment (inner join - intersection):")
print(f"  Both datasets now have {len(ds1_aligned.coords['time'])} time steps")

# Example 2: Merging datasets
# Merge two datasets (combines variables)
# This is useful when you have different variables in different files
ds_merged = xr.merge([ds_period1, ds_period2], join='outer')  # 'outer' = union
print(f"\nMerged dataset time range: {ds_merged.coords['time'].values[0]} to {ds_merged.coords['time'].values[-1]}")
print(f"  Total time steps: {len(ds_merged.coords['time'])}")

# Example 3: Concatenating along a dimension
# Combine datasets along time dimension
ds_concatenated = xr.concat([ds_period1, ds_period2], dim='time')
print(f"\nConcatenated dataset time steps: {len(ds_concatenated.coords['time'])}")
print(f"  Should equal sum of both periods: {len(ds_period1.coords['time']) + len(ds_period2.coords['time'])}")

print("\n💡 Key functions:")
print("   - xr.align(): Align datasets to have matching coordinates")
print("   - xr.merge(): Combine datasets with different variables")
print("   - xr.concat(): Combine datasets along a dimension")


**Now You Try**: After learning about alignment operations, try:
- Splitting your dataset into two time periods and aligning them
- Merging two different selections of your dataset
- Concatenating data from different locations along the location dimension


In [ ]:
# Compare mean density across locations
mean_density_by_location = ds['density'].mean(dim=['time', 'layer'])
print("Mean density by location:")
for i, loc in enumerate(ds.coords['location'].values):
    mean_dens = mean_density_by_location.isel(location=i).values
    print(f"  Location {loc}: {mean_dens:.1f} kg/m³")

# Compare snow height evolution
print("\nSnow height by location (last time step):")
for i, loc in enumerate(ds.coords['location'].values):
    hs_loc = ds['HS'].isel(location=i, slope=0, realization=0, time=-1)
    print(f"  Location {loc}: {hs_loc.values:.2f} m")


**Now You Try**: After running the temporal analysis, try:
- Computing a different rolling window (try 3-day, 14-day, or 30-day averages)
- Calculating the rate of change for a different variable (e.g., density or temperature)
- Finding the time period with the fastest snow height increase


## Part 4: Advanced Temporal Analysis

Time series analysis helps understand how the snowpack evolves over time.


In [ ]:
# Extract snow height time series
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
times = ds.coords['time'].values

# 7-day rolling average (smoothing)
hs_7day = hs_series.rolling(time=7, center=True).mean()

# Rate of change
hs_rate = hs_series.diff(dim='time')

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
ax1.plot(times, hs_series.values, 'b-', alpha=0.5, label='Original')
ax1.plot(times, hs_7day.values, 'r-', linewidth=2, label='7-day rolling average')
ax1.set_xlabel('Time')
ax1.set_ylabel('Snow Height (m)')
ax1.set_title('Snow Height Time Series')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(times[1:], hs_rate.values[1:], 'g-', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Time')
ax2.set_ylabel('Rate of Change (m/time step)')
ax2.set_title('Snow Height Rate of Change')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean rate of change: {hs_rate.mean().values:.4f} m/time step")


## Summary

In this notebook, you learned:

1. **Density gradients**: Identifying weak layers through density changes with depth
2. **Temperature gradients**: Understanding faceting conditions (gradients > 10 K/m indicate weak layer formation)
3. **Multi-location comparison**: Comparing statistics across different locations
4. **Temporal analysis**: Using rolling averages to smooth time series and calculating rates of change

## Key Techniques

- **Gradients**: `ds['variable'].diff(dim='layer')` for layer-to-layer differences
- **Temperature gradient**: `temp_diff / z_diff` to get gradient per unit depth
- **Rolling windows**: `ds.rolling(time=7).mean()` for smoothing time series
- **Rate of change**: `ds.diff(dim='time')` to see how values change over time
- **Multi-location stats**: `.mean(dim=['time', 'layer'])` to aggregate across dimensions

## Next Steps

- **05_working_with_custom_data.ipynb**: Load and analyze your own files
- **06_extending_xsnow.ipynb**: Create custom analysis functions

## Exercises

1. Compute temperature gradient and identify layers with gradients > 10 K/m
2. Calculate a 7-day rolling average of snow height for a different location
3. Compare mean density profiles across all locations in your dataset
4. Plot the rate of change of temperature at a specific depth over time
